In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv('./df.csv')
df.drop(axis=1,columns=['fullVisitorId'],inplace=True)
df['transactionRevenue'] =  pd.to_numeric(df['transactionRevenue'])
y = df['transactionRevenue']
y = y.to_frame(name='transactionRevenue')
df.drop(axis=1,columns=['transactionRevenue','date'],inplace=True)
le = LabelEncoder()
for i in range(df.shape[1]) :
    if(df[df.columns[i]].dtypes == 'O') :
        le.fit(df[df.columns[i]]);
        df[df.columns[i]] = le.transform(df[df.columns[i]]) 
x = df
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=123,shuffle=False)

In [ ]:
# normalization 

In [ ]:
# feature selection

In [ ]:
# imbalance handling

In [ ]:
from sklearn.tree import DecisionTreeRegressor
regr_2 = DecisionTreeRegressor(max_depth=5)
regr_2.fit(x_train, y_train)
y_pred = regr_2.predict(x_test)

In [ ]:
y_add = y_test + 1
y_log = np.log(y_add)
y_pred_log = np.log(y_pred)
np.sqrt(mean_squared_error(y_log, y_pred_log))

In [ ]:
y_pred
y_pred_log

In [ ]:
import json
def load_df(csv_path='./original/test.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    df = pd.read_csv(csv_path, dtype={'fullVisitorId': 'str'}, nrows=nrows)
    for column in JSON_COLUMNS:
        df = df.join(pd.DataFrame(df.pop(column).apply(pd.io.json.loads).values.tolist(), index=df.index))
    return df

df = load_df()
df.drop(columns='adwordsClickInfo',axis=1,inplace=True)
keep_columns = df.columns[df.nunique()>1]
df = df.loc[:,keep_columns].copy()
def add_time_features(df):
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='ignore')
    df['year'] = df['date'].apply(lambda x: x.year)
    df['month'] = df['date'].apply(lambda x: x.month)
    df['day'] = df['date'].apply(lambda x: x.day)
    df['weekday'] = df['date'].apply(lambda x: x.weekday())
    return df
df = add_time_features(df)
df['pageviews'] = df['pageviews'].convert_objects(convert_numeric=True)
df.drop(columns=['visitStartTime'],inplace=True)
df.drop(columns=['sessionId','visitId'],inplace=True)
df.drop(columns=['isMobile','deviceCategory','networkDomain','pageviews','medium','campaign','referralPath','keyword'
                   ,'adContent'],inplace=True)
df.drop(columns=['city','continent','metro','region','subContinent'],inplace=True)
df.drop_duplicates(subset='fullVisitorId',inplace=True)
fullVisitorId = df['fullVisitorId']
df.drop(axis=1,columns='fullVisitorId',inplace=True)
df.drop(axis=1,columns=['date'],inplace=True)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in range(df.shape[1]) :
    if(df[df.columns[i]].dtypes == 'O') :
        le.fit(df[df.columns[i]])
        df[df.columns[i]] = le.transform(df[df.columns[i]]) 
ans = regr_2.predict(df)
ans.shape

In [ ]:
ans_df = pd.DataFrame(data=fullVisitorId)
ans_df.columns = ['fullVisitorId']
temp = pd.DataFrame()
ans_df['PredictedLogRevenue'] = ans
ans_df['PredictedLogRevenue']=ans_df['PredictedLogRevenue'].apply(lambda x: 0 if x == 0 else np.log(x))
ans_df.to_csv('./ans.csv',index=False)

In [ ]:
ans_df['fullVisitorId'].describe()

In [ ]:
ans_df['PredictedLogRevenue'].describe()